In [ ]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('../desdeo_emo')))

In [ ]:
import plotly.graph_objects as go
import numpy as np

from desdeo_problem.Variable import variable_builder
from desdeo_problem.Objective import ScalarObjective
from desdeo_problem.Problem import MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs.NSGAIII import NSGAIII
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.othertools.plotlyanimate import animate_init_, animate_next_

## Coello MOP7

### Definition
![Definition](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/examples/notebook_extras/MOP7.png)

### Pareto set and front
![Front](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/examples/notebook_extras/MOP7_PF.png)

## Define objective functions

In [ ]:
def f_1(x):
    term1 = ((x[:,0] - 2) ** 2) / 2
    term2 = ((x[:,1] + 1) ** 2) / 13
    return term1 + term2 + 3

def f_2(x):
    term1 = ((x[:, 0] + x[:, 1] - 3) ** 2) / 36
    term2 = ((-x[:, 0] + x[:, 1] + 2) ** 2) / 8
    return term1 + term2 - 17

def f_3(x):
    term1 = ((x[:, 0] + (2 * x[:, 1]) - 1) ** 2) / 175
    term2 = ((-x[:, 0] + 2* x[:, 1]) ** 2) / 17
    return term1 + term2 - 13

Note that the expected input `x` is two dimensional. It should be a 2-D numpy array.

## Create Variable objects

In [ ]:
help(variable_builder)

In [ ]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-400, -400],
                             upper_bounds=[400, 400])
list_vars

## Create Objective objects

In [ ]:
f1 = ScalarObjective(name='f1', evaluator=f_1)
f2 = ScalarObjective(name='f2', evaluator=f_2)
f3 = ScalarObjective(name='f3', evaluator=f_3)
list_objs = [f1, f2, f3]

## Create the problem object

In [ ]:
problem = MOProblem(variables=list_vars, objectives=list_objs)

## Using the EAs

Pass the problem object to the EA, pass parameters as arguments if required.

In [ ]:
help(RVEA)

In [ ]:
evolver = NSGAIII(problem,
                  n_iterations=3,
                  n_gen_per_iter=20,
                  population_size=500)

In [ ]:
while evolver.continue_evolution():
    evolver.run_interruption()
    evolver.iterate()

## Extracting optimized decision variables and objective values

In [ ]:
individuals = evolver.population.individuals

fig1 = go.Figure(data=go.Scatter(x=individuals[:,0], y=individuals[:,1], mode="markers"))
fig1

In [ ]:
objectives = evolver.population.objectives
fig2 = go.Figure(data=go.Scatter3d(x=objectives[:,0],
                                   y=objectives[:,1],
                                   z=objectives[:,2],
                                   mode="markers",
                                   marker_size=1))
fig2

## Coello MOP5

### Definition
![Definition](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/examples/notebook_extras/MOP5.png)

### Pareto set and front
![Front](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/examples/notebook_extras/MOP5_PF.png)

In [ ]:
def f_4(x):
    term = x[:, 0]**2 + x[:, 1]**2
    return 0.5*term + np.sin(term)

def f_5(x):
    term1 = ((3*x[:, 0] - 2*x[:,1] + 4)**2)/8
    term2 = ((x[:, 0] - x[:,1] + 1)**2)/27
    return term1 + term2 + 15

def f_6(x):
    term = x[:, 0]**2 + x[:, 1]**2
    return (1/(term + 1)) - 1.1 * np.exp(-term)

In [ ]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-30, -30],
                             upper_bounds=[30, 30])

In [ ]:
f1 = ScalarObjective(name='f1', evaluator=f_4)
f2 = ScalarObjective(name='f2', evaluator=f_5)
f3 = ScalarObjective(name='f3', evaluator=f_6)

In [ ]:
problem = MOProblem(variables=list_vars, objectives=[f1, f2, f3])

In [ ]:
evolver = RVEA(problem)

In [ ]:
figure = animate_init_(evolver.population.objectives, filename="MOP5.html")

In [ ]:
while evolver.continue_evolution():
    evolver.run_interruption()
    evolver.iterate()
    figure = animate_next_(
        evolver.population.objectives,
        figure,
        filename="MOP5.html",
        generation=evolver._iteration_counter-2,
    )

## Interaction in EAs

The reference point method has been implemented.

In [ ]:
help(test_problem_builder)

In [ ]:
problem = test_problem_builder(name="DTLZ1", n_of_variables=30, n_of_objectives=3)

In [ ]:
evolver = NSGAIII(problem, interact=True, n_iterations=5, n_gen_per_iter=400)
figure = animate_init_(evolver.population.objectives, filename="dtlz1.html")

In [ ]:
while evolver.continue_evolution():
    evolver.run_interruption()
    evolver.iterate()
    figure = animate_next_(
        evolver.population.objectives,
        figure,
        filename="dtlz1.html",
        generation=evolver._iteration_counter-2,
    )